<h1>Cluster the neighborhood data</h1>

<h2>1) </h2>

In [1]:
%store -r Toronto_latlng
Toronto_latlng.head()

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [7]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\soraj\Anaconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    conda-4.6.10               |           py37_0         914 KB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         999 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0     conda-forge
    geopy:         1.19.0-py_0   conda-forge

The following packages will be UPDATED:

    conda:         4.5.12-py37_0             --> 4.6.10-py37_0 conda-forge


Preparing transaction: ...working... done
Verifying transaction: ...working... done




==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.9

Please update conda by running

    $ conda update -n base -c defaults conda



geographiclib-1.49   | 32 KB     |            |   0% 
geographiclib-1.49   | 32 KB     | ########## | 100% 

conda-4.6.10         | 914 KB    |            |   0% 
conda-4.6.10         | 914 KB    | ##3        |  24% 
conda-4.6.10         | 914 KB    | ########2  |  82% 
conda-4.6.10         | 914 KB    | #########2 |  92% 
conda-4.6.10         | 914 KB    | ########## | 100% 

geopy-1.19.0         | 53 KB     |            |   0% 
geopy-1.19.0         | 53 KB     | ########## | 100% 


In [86]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: ...working... done

# All requested packages already installed.



In [8]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [85]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Toronto_latlng['latitude'], Toronto_latlng['longitude'], Toronto_latlng['Borough'], Toronto_latlng['Neighbourhood']):
   label = '{}, {}'.format(neighborhood, borough)
   label = folium.Popup(label, parse_html=True)
   folium.CircleMarker(
       [45, -10],
       radius=5,
       popup=label,
       color='blue',
       fill=True,
       fill_color='#3186cc',
       fill_opacity=0.7,
       parse_html=False).add_to(map_Toronto)  

#folium.CircleMarker(location=[43.80, -79.194], radius=5, line_color='#3186cc', fill_color='#3186cc').add_to(map_Toronto) 
#folium.CircleMarker([43.80, -79.194],radius=5,parse_html=False).add_to(map_Toronto)  
# add markers to map

map_Toronto



In [65]:
CLIENT_ID = 'ZLNCVVDMUTSIAICKT103POACTFZMR4COUZMCMB4UUN5MYA3X' # your Foursquare ID
CLIENT_SECRET = 'SCRJ41NA15ZHMBPSEMB115J3SBV33QTBTMUQ25CZJOBWEBKH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [54]:

neighborhood_latitude = Toronto_latlng.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_latlng.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = Toronto_latlng.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


In [55]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZLNCVVDMUTSIAICKT103POACTFZMR4COUZMCMB4UUN5MYA3X&client_secret=SCRJ41NA15ZHMBPSEMB115J3SBV33QTBTMUQ25CZJOBWEBKH&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [52]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca57b55351e3d25e1dca14f'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 45.0045000045, 'lng': -9.993647914860684},
   'sw': {'lat': 44.9954999955, 'lng': -10.006352085139316}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [56]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [58]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: "None of [['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']] are in the [columns]"